In [1]:
import csv
import random
import math

In [3]:
# creating the function to load the dataset.
def LoadDataset(filename):
    dataset = []
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            dataset.append(row)
    return dataset

In [5]:

# then split the dataset into train, dev, and test sets
def split_train_dev_eval(dataset, split_ratio):
    train_size = int(len(dataset) * split_ratio) # split the train according to input ratio
    train_dataset = []
    devandeval = dataset[:]
    while len(train_dataset) < train_size:
        index = random.randrange(len(devandeval))
        train_dataset.append(devandeval.pop(index))
    return [train_dataset, devandeval, devandeval] #return list contains the three parts.

In [6]:
# function that calculates the word occurence in all reviews
def word_occurrences(dataset, vocabulary):
    WordOccurrences = {}
    for word in vocabulary:
        WordOccurrences[word] = 0
        for document in dataset:
            if word in document[0].split():
                WordOccurrences[word] += 1
    return WordOccurrences

In [ ]:

# function that calculates the probability of each class (fresh or rotten)
def ClassProbabilities(dataset):

    class_probabilities = {}
    
    for document in dataset:
        sentiment = document[0]
        if sentiment == "Freshness": # ignoring the excess lables from the dataset.
            pass
        else:

            if sentiment not in class_probabilities:
                class_probabilities[sentiment] = 1 
            else:
                class_probabilities[sentiment] += 1 #counting the number of reviews by adding ones in each loop trun.
    total_documents = len(dataset) #the number of all reviews
    
    for sentiment in class_probabilities:
        class_probabilities[sentiment] /= total_documents
        
        
    return class_probabilities #returns dictionary of both classes probabilities.